In [1]:
import tensorflow as tf
import os, logging
tf.logging.set_verbosity(tf.logging.ERROR)
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from datetime import datetime

tf.compat.v1.set_random_seed(100)
np.random.seed(100)

import keras.metrics
from keras import backend as K
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# argparser for training the model
import argparse

def parse_args():
    parser = argparse.ArgumentParser(add_help=False)

    # dataset
    parser.add_argument('--dataset', type=str, default='physionet2012', choices=['physionet2012', 'MIMIC3'],
                        help="dataset")
    parser.add_argument('--application', type=str, default='inhos_mortality', choices=['inhos_mortality'],
                        help="dataset")
    parser.add_argument('--folds', type=list, default=[0],
                        help='folds id')
    parser.add_argument('--ffill', action='store_true',default=True,
                        help='data filling， ffill or None')
    parser.add_argument('--standardization', action='store_false',default=True,
                        help='standardization for the training dataset')
    parser.add_argument('--data_clip', action='store', default=False,
                        help='data clipping: decide the maximun and minimun value of the training dataset')
    parser.add_argument('--data_clip_min', type=float, default=-1*float('inf'),
                        help='data clipping: minimun value of the training dataset')
    parser.add_argument('--data_clip_max', type=float, default=float('inf'),
                        help='data clipping: maximun value of the training dataset')
    parser.add_argument('--dataset_mode', type=str, default='regular', choices=['regular'],
                        help="regular or irregular")
    parser.add_argument('--ffill_steps', type=int, default=48,
                        help='data filling steps')
    parser.add_argument('--max_timesteps', type=int, default=48,
                        help='Time series of at most # time steps are used. Default: 48.')

    # model
    parser.add_argument('--model', type=str, default='EDUTEM', choices=['EDUTEM','EDUTEM_t','EDUTEM_f'],
                        help="model")
    parser.add_argument('--embed_dim', type=int, default=24,
                        help="embed_dim")
    parser.add_argument('--hidden_dim', type=int, default=64,
                        help="hidden_dim")
    parser.add_argument('--clip_min', type=float, default=-3.0,
                        help="clip_min")
    parser.add_argument('--clip_max', type=float, default=3.0,
                        help="clip_max")
    parser.add_argument('--compress_dim', type=int, default=4,
                        help='compress_dim for interaction features')

    # mode
    parser.add_argument('--mode', type=str, default="train",
                        help="train or eval")
    parser.add_argument('--model_path', type=str, default="",
                        help="existing model path")
    parser.add_argument('--debug', action='store_true',default=False,
                        help='for backup')
    parser.add_argument('--random', action='store_true',default=False,
                        help='for backup')

    # training configuration
    parser.add_argument('--epoch', type=int, default=200,
                        help='Number of epochs.')
    parser.add_argument('--batch_size', type=int, default=64,
                        help='Batch size.')
    parser.add_argument('--lr', type=float, default=0.001,
                        help='Learning rate.')
    parser.add_argument('--weight_decay', type=float, default=1e-7,
                        help='decrease overfitting.')
    parser.add_argument('--patience', type=int, default=10,
                        help='patience for earlystopping.')
    parser.add_argument('--gpu', type=str, default="0,1,2",
                        help='GPU ids')

    return parser.parse_args(args=[])

args = parse_args()
args

Namespace(application='inhos_mortality', batch_size=64, clip_max=3.0, clip_min=-3.0, compress_dim=4, data_clip=False, data_clip_max=inf, data_clip_min=-inf, dataset='physionet2012', dataset_mode='regular', debug=False, embed_dim=24, epoch=200, ffill=True, ffill_steps=48, folds=[0], gpu='0,1,2', hidden_dim=64, lr=0.001, max_timesteps=48, mode='train', model='EDUTEM', model_path='', patience=10, random=False, standardization=True, weight_decay=1e-07)

In [3]:
# set GPU usage for tensorflow backend
if K.backend() == 'tensorflow':
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [4]:
if args.dataset == "physionet2012":
    from dataset.physionet2012.DataLoader import physonet2012_DataLoader
elif args.dataset == "MIMIC3":
    from dataset.MIMIC3.DataLoader import MIMIC3_DataLoader
else:
    raise FileNotFoundError("No such dataset: %s" % args.dataset)

In [5]:
# Recoders can record the results of the model in each epoch

# the results in the first three lines show the performance of the model which has the best bceloss on the validation set.
# the results in the second three lines show the performance of the model which has the best AUC-ROC on the validation set.
# the results in the last three lines show the performance of the model which has the best AUC-PR on the validation set.
import pandas as pd

class Recoder():
    def __init__(self,label,features):
        self.count = 0
        self.features = features
        self.data = {
            f: [] for f in features
        }
        self.label = label

    def insert(self,result):
        self.count += 1
        for i in self.features:
            self.data[i].append(result[i])
        return result

    def best(self,feature,mode=max):
        best_feature = mode(self.data[feature])
        index_best = self.data[feature].index(best_feature)
        return index_best

    def select_index(self,index):
        return {
            f:"%.4f" % self.data[f][index] for f in self.features
        }


class Recoders():
    def __init__(self,output_path,title,mode=2):
        self.features = ["bceloss","auroc","auprc","acc"] # ,"rec1","prec1"
        self.feature_size = len(self.features)
        if mode == 3:
            self.sets = ["train","valid","test"]
        elif mode == 2:
            self.sets = ["train", "valid"]
        self.recoders = {}
        for set in self.sets:
            self.recoders[set] = Recoder(set,self.features)
        self.count = 0
        self.output_path = output_path
        self.title = title
        self.min_score = 100


    def init_score(self,score):
        self.min_score = score

    def lower_score(self,score,delta=0):
        if score < self.min_score+delta:
            self.count = 0
            self.min_score = score
            return True
        else:
            self.count += 1
            return False

    def clean(self):
        for set in self.sets:
            self.recoders[set] = Recoder(set,self.features)
        self.count = 0
        self.min_score = 100

    def insert(self,label,result,verbose=1):
        self.recoders[label].insert(result)
        if verbose:
            log_r = "[%s] " % label
            for f in self.features:
                log_r += "%s:%.4f " % (f, result[f])
            logging.info(log_r)
            # print(log_r)

    def record_to_csv(self,i_fold=0):
        result = pd.DataFrame()
        for roc in ['bceloss','auprc','auroc']:
            tem = pd.DataFrame(columns=['Epoch']+self.features,index=self.sets)
            if roc == 'bceloss':
                func = min
            else:
                func = max
            best_epoch = self.recoders["valid"].best(roc, func)
            for set in self.sets:
                if set == "train":
                    tem.loc['train', 'Epoch'] = self.recoders["train"].count
                elif set == "valid":
                    tem.loc['valid', 'Epoch'] = best_epoch+1
                elif set == "test":
                    tem.loc['test', 'Epoch'] = self.recoders["test"].best(roc, func)+1
            for dataset in self.sets:
                for f in self.features:
                    tem.loc[dataset,f] = self.recoders[dataset].select_index(best_epoch)[f]
            result = pd.concat([result,tem],0)
        logging.info(result)
        result.to_csv(self.output_path + self.title+'%d.csv'%i_fold)


In [6]:
from sklearn import metrics

EPS = 1e-10

# binary_eval can evaluate binary prediction results
def binary_eval(y_true, predictions, verbose=1):
    predictions = np.array(predictions)
    if len(predictions.shape) == 1:
        predictions = np.stack([1 - predictions, predictions]).transpose((1, 0))

    cf = metrics.confusion_matrix(y_true, predictions.argmax(axis=1))
    if verbose:
        logging.info("confusion matrix:")
        logging.info(cf)
    cf = cf.astype(np.float32)

    acc = (cf[0][0] + cf[1][1]) / np.sum(cf)
    prec0 = cf[0][0] / (cf[0][0] + cf[1][0] + EPS)
    prec1 = cf[1][1] / (cf[1][1] + cf[0][1] + EPS)
    rec0 = cf[0][0] / (cf[0][0] + cf[0][1] + EPS)
    rec1 = cf[1][1] / (cf[1][1] + cf[1][0] + EPS)
    auroc = metrics.roc_auc_score(y_true, predictions[:, 1])

    (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, predictions[:, 1])
    auprc = metrics.auc(recalls, precisions)
    minpse = np.max([min(x, y) for (x, y) in zip(precisions, recalls)])

    bce = []
    for i in range(len(y_true)):
        bce.append(-y_true[i] * np.log(predictions[i,1] + EPS) - (1 - y_true[i]) * np.log(1 - predictions[i,1] + EPS))
    bceloss = np.mean(bce)

    if verbose:
        logging.info("accuracy = {}".format(acc))
        logging.info("precision class 0 = {}".format(prec0))
        logging.info("precision class 1 = {}".format(prec1))
        logging.info("recall class 0 = {}".format(rec0))
        logging.info("recall class 1 = {}".format(rec1))
        logging.info("AUC of ROC = {}".format(auroc))
        logging.info("AUC of PRC = {}".format(auprc))
        logging.info("min(+P, Se) = {}".format(minpse))
        logging.info("BCEloss = {}".format(bceloss))

    return {"acc": acc,
            "prec0": prec0,
            "prec1": prec1,
            "rec0": rec0,
            "rec1": rec1,
            "auroc": auroc,
            "auprc": auprc,
            "minpse": minpse,
            "bceloss": bceloss,
            }

In [7]:
from keras import backend as K
from keras.layers import Activation, Dense, Dropout, Input, Masking
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import GRU
from keras.layers.wrappers import Bidirectional
from keras.models import load_model, Model
from keras.regularizers import l2

from model.keras_layers import EDUTEM_f,EDUTEM_t


def EDUTEM_net(info_dim,input_dim,output_dim,time_dim,args):
    input_info = Input(shape=(info_dim,))
    input_stime = Input(shape=(time_dim,))
    input_tdata = Input(shape=(time_dim, input_dim))
    input_tmask = Input(shape=(time_dim, input_dim))

    input_list = [input_info, input_stime, input_tdata, input_tmask]

    EDUTEM_f_layer = EDUTEM_f(input_dim=input_dim,
                         embedding_dim=args.embed_dim,
                         time_step=time_dim,
                         compress_dim=args.compress_dim,
                         clip_min=args.clip_min,
                         clip_max=args.clip_max,
                         )

    EDUTEM_t_layer = EDUTEM_t(time_step=time_dim,
                         hidden_dim=args.hidden_dim
                         )

    gru_layer = GRU(units=args.hidden_dim,
                    activation='sigmoid',
                    return_sequences=True,
                    )

    x = EDUTEM_f_layer([input_tdata, input_tmask])
    x = gru_layer(x)
    x = EDUTEM_t_layer(x)

    x = Dense(units=64, activation='relu',kernel_regularizer=l2(1e-4))(x)
    x = Dense(output_dim, activation='sigmoid')(x)
    output_list = [x]
    model = Model(inputs=input_list, outputs=output_list)
    return model


def EDUTEM_t_net(info_dim,input_dim,output_dim,time_dim,args):
    input_info = Input(shape=(info_dim,))
    input_stime = Input(shape=(time_dim,))
    input_tdata = Input(shape=(time_dim, input_dim))
    input_tmask = Input(shape=(time_dim, input_dim))

    input_list = [input_info, input_stime, input_tdata, input_tmask]

    EDUTEM_t_layer = EDUTEM_t(time_step=time_dim,
                         hidden_dim=args.hidden_dim
                         )

    gru_layer = GRU(units=args.hidden_dim,
                    activation='sigmoid',
                    return_sequences=True,
                    )

    x = gru_layer(input_tdata)
    x = EDUTEM_t_layer(x)

    x = Dense(output_dim, activation='sigmoid')(x)
    output_list = [x]
    model = Model(inputs=input_list, outputs=output_list)
    return model

def EDUTEM_f_net(info_dim,input_dim,output_dim,time_dim,args):
    input_info = Input(shape=(info_dim,))
    input_stime = Input(shape=(time_dim,))
    input_tdata = Input(shape=(time_dim, input_dim))
    input_tmask = Input(shape=(time_dim, input_dim))

    input_list = [input_info, input_stime, input_tdata, input_tmask]

    EDUTEM_f_layer = EDUTEM_f(input_dim=input_dim,
                         embedding_dim=args.embed_dim,
                         time_step=time_dim,
                         compress_dim=args.compress_dim,
                         clip_min=args.clip_min,
                         clip_max=args.clip_max,
                         )

    gru_layer = GRU(units=args.hidden_dim,
                    activation='sigmoid',
                    return_sequences=False,
                    )

    x = EDUTEM_f_layer([input_tdata, input_tmask])
    x = gru_layer(x)

    x = Dense(output_dim, activation='sigmoid')(x)
    output_list = [x]
    model = Model(inputs=input_list, outputs=output_list)
    return model

In [8]:
class main():
    def __init__(self):
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
        self.log_path = self.set_log_path(timestamp)

        handlers = [logging.FileHandler(self.log_path + 'log_{}.txt'.format(timestamp), mode='w'), logging.StreamHandler()]
        logging.basicConfig(level=logging.INFO, datefmt='%m-%d-%y %H:%M', format='%(asctime)s:%(message)s', handlers=handlers)

        logging.info('Timestamp: {}'.format(timestamp))
        logging.info('Arguments:%s' % args)

        self.sets = ['train', 'valid', 'test']
        # sets = ['train', 'valid']

        self.recoders = Recoders(self.log_path, timestamp, len(self.sets))

        for i_fold in args.folds:
            print('{}-th fold...'.format(i_fold))
            self.dataset = self.load_dataset(i_fold)
            self.model = self.load_model()
            if args.mode == "train":
                self.train(i_fold)
            self.eval(i_fold)

    def set_log_path(self,timestamp):
        log_path = os.getcwd() + "/logs/"
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        if args.debug:
            log_path += "/%s_%s_debug/" % (args.model, timestamp)
            if not os.path.exists(log_path):
                os.mkdir(log_path)
            return log_path

        log_path += "/%s/"% args.dataset
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        log_path += "/%s/"% args.application
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        log_path += "/%s/"% args.model
        if not os.path.exists(log_path):
            os.mkdir(log_path)

        log_path += "/%s_%s/" % (args.model, timestamp)
        if args.random:
            log_path = log_path[:-1]+"_random/"
        if not os.path.exists(log_path):
            os.mkdir(log_path)
        return log_path

    def load_dataset(self, i_fold):
        logging.info("[*] Loading dataset: %s" % args.dataset)
        if args.dataset == "physionet2012":
            dataset = physonet2012_DataLoader(args=args, fold_id=i_fold, label=args.application, debug=args.debug)
        elif args.dataset == "MIMIC3":
            dataset = MIMIC3_DataLoader(args=args, fold_id=i_fold, label=args.application, debug=args.debug)
        else:
            raise NameError(args.dataset)

        train_len = dataset.get_subset_size("train")
        valid_len = dataset.get_subset_size("valid")
        if len(self.sets) == 3:
            test_len = dataset.get_subset_size("test")
        else:
            test_len = 0
        logging.info("train: %d valid: %d test: %d" % (train_len, valid_len, test_len))
        return dataset


    def load_model(self):
        if args.dataset_mode == "regular":
            if args.model == "EDUTEM":
                model = EDUTEM_net(info_dim=self.dataset.info_dim, input_dim=self.dataset.input_dim,output_dim=self.dataset.output_dim,
                                     time_dim=48, args=args)
            elif args.model == "EDUTEM_t":
                model = EDUTEM_t_net(info_dim=self.dataset.info_dim, input_dim=self.dataset.input_dim,output_dim=self.dataset.output_dim,
                                     time_dim=48, args=args)
            elif args.model == "EDUTEM_f":
                model = EDUTEM_f_net(info_dim=self.dataset.info_dim, input_dim=self.dataset.input_dim,output_dim=self.dataset.output_dim,
                                     time_dim=48, args=args)
            else:
                raise NotImplementedError("No such model in regular mode: %s" % (args.model))

        model.summary(print_fn=logging.info)
        return model


    def eval_epoch(self, epoch, dataset, model, i_fold,save=True):
        true_y_list = [
            dataset.sub_label("train"),dataset.sub_label("valid")
        ]
        pred_y_list = [
            model.predict_generator(dataset.get_generator("train", shuffle=False, batch_size=args.batch_size, return_whole=False),
                                    steps=dataset.sub_steps("train", batch_size=args.batch_size)),
            model.predict_generator(dataset.get_generator("valid", shuffle=False, batch_size=args.batch_size, return_whole=False),
                                    steps=dataset.sub_steps("valid", batch_size=args.batch_size)),
        ]
        if len(self.sets) == 3:
            true_y_list.append(dataset.sub_label("test"))
            pred_y_list.append(model.predict_generator(dataset.get_generator("test", shuffle=False, batch_size=args.batch_size, return_whole=False),
                                                       steps=dataset.sub_steps("test", batch_size=args.batch_size)))

        logging.info("[%s Epoch %d]" % (args.model, epoch))
        for index,dataset in enumerate(self.sets):
            result = binary_eval(np.reshape(true_y_list[index],(-1)), np.reshape(pred_y_list[index],(-1)), 0)
            self.recoders.insert(dataset, result)
            if dataset == "valid" and self.recoders.lower_score(result['bceloss']) and save:
                logging.info("[*] model_%d.h5 saved "%i_fold)
                model.save(self.log_path+'model_%d.h5'%i_fold)



    def train(self, i_fold):
        optimizer = Adam(learning_rate=args.lr,decay=0.0)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=[keras.metrics.BinaryCrossentropy()])

        self.model.fit_generator(
            generator=self.dataset.get_generator("train", shuffle=True, batch_size=args.batch_size, return_whole=True),
            steps_per_epoch=self.dataset.sub_steps("train", batch_size=args.batch_size),
            epochs=args.epoch,
            verbose=1,
            validation_data=self.dataset.get_generator("valid", shuffle=False, batch_size=args.batch_size, return_whole=True),
            validation_steps=self.dataset.sub_steps("valid", batch_size=args.batch_size),
            callbacks=[
                LambdaCallback(on_epoch_end=lambda epoch, logs: self.eval_epoch(epoch + 1, self.dataset, self.model, i_fold)),
                EarlyStopping(monitor="val_binary_crossentropy",patience=args.patience,mode="min")
                # reduce_lr,
            ]
        )
        self.recoders.record_to_csv(i_fold)
        self.recoders.clean()


    def eval(self, i_fold):
        if args.mode == "eval":
            if args.model_path != "" and os.path.exists(args.model_path):
                self.model.load_weights(args.model_path)
            else:
                raise FileNotFoundError("There is no such file for the model: %s" % args.model_path)

        elif args.mode == "train":
            self.model.load_weights(self.log_path+'model_%d.h5'%i_fold)

        logging.info("[Eval]")
        self.eval_epoch(0, self.dataset, self.model, i_fold, False)


In [9]:
main()

02-08-21 11:40:Timestamp: 20210208114032
02-08-21 11:40:Arguments:Namespace(application='inhos_mortality', batch_size=64, clip_max=3.0, clip_min=-3.0, compress_dim=4, data_clip=False, data_clip_max=inf, data_clip_min=-inf, dataset='physionet2012', dataset_mode='regular', debug=False, embed_dim=24, epoch=200, ffill=True, ffill_steps=48, folds=[0], gpu='0,1,2', hidden_dim=64, lr=0.001, max_timesteps=48, mode='train', model='EDUTEM', model_path='', patience=10, random=False, standardization=True, weight_decay=1e-07)
02-08-21 11:40:[*] Loading dataset: physionet2012
02-08-21 11:40:fold path: inhos_mortality_folds.npz
02-08-21 11:40:train: [150491 143937 148710 140049 156691]
  0%|          | 23/9600 [00:00<00:42, 227.48it/s]

0-th fold...


100%|██████████| 1200/1200 [00:04<00:00, 282.08it/s]
02-08-21 11:41:train: 9600 valid: 1200 test: 1200
02-08-21 11:41:Model: "model_1"
02-08-21 11:41:__________________________________________________________________________________________________
02-08-21 11:41:Layer (type)                    Output Shape         Param #     Connected to                     
02-08-21 11:41:==================================================================================================
02-08-21 11:41:input_3 (InputLayer)            (None, 48, 37)       0                                            
02-08-21 11:41:__________________________________________________________________________________________________
02-08-21 11:41:input_4 (InputLayer)            (None, 48, 37)       0                                            
02-08-21 11:41:__________________________________________________________________________________________________
02-08-21 11:41:edutem_f_1 (EDUTEM_f)           (None, 48, 148)     

Epoch 1/200
150/150 [==============================] - 16s 107ms/step - loss: 0.4006 - binary_crossentropy: 0.3946 - val_loss: 0.4489 - val_binary_crossentropy: 0.3632


02-08-21 11:41:[EDUTEM Epoch 1]
02-08-21 11:41:[train] bceloss:0.3636 auroc:0.7511 auprc:0.3603 acc:0.8585 
02-08-21 11:41:[valid] bceloss:0.3632 auroc:0.7536 auprc:0.3953 acc:0.8583 
02-08-21 11:41:[*] model_0.h5 saved 
02-08-21 11:41:[test] bceloss:0.3715 auroc:0.7190 auprc:0.3733 acc:0.8592 


...

02-08-21 11:57:[EDUTEM Epoch 41]
02-08-21 11:57:[train] bceloss:0.2537 auroc:0.9015 auprc:0.6634 acc:0.8956 
02-08-21 11:57:[valid] bceloss:0.3001 auroc:0.8599 auprc:0.5649 acc:0.8775 
02-08-21 11:57:[test] bceloss:0.2816 auroc:0.8849 auprc:0.5734 acc:0.8883 


Epoch 42/200
150/150 [==============================] - 15s 101ms/step - loss: 0.2605 - binary_crossentropy: 0.2586 - val_loss: 0.4436 - val_binary_crossentropy: 0.2925


02-08-21 11:57:[EDUTEM Epoch 42]
02-08-21 11:57:[train] bceloss:0.2497 auroc:0.9025 auprc:0.6674 acc:0.8973 
02-08-21 11:57:[valid] bceloss:0.2925 auroc:0.8579 auprc:0.5517 acc:0.8800 
02-08-21 11:57:[test] bceloss:0.2766 auroc:0.8812 auprc:0.5651 acc:0.8842 


Epoch 43/200
150/150 [==============================] - 15s 102ms/step - loss: 0.2558 - binary_crossentropy: 0.2538 - val_loss: 0.4315 - val_binary_crossentropy: 0.2970


02-08-21 11:57:[EDUTEM Epoch 43]
02-08-21 11:57:[train] bceloss:0.2488 auroc:0.9051 auprc:0.6762 acc:0.8999 
02-08-21 11:57:[valid] bceloss:0.2970 auroc:0.8536 auprc:0.5463 acc:0.8808 
02-08-21 11:57:[test] bceloss:0.2752 auroc:0.8837 auprc:0.5701 acc:0.8825 
02-08-21 11:57:      Epoch bceloss   auroc   auprc     acc
train    43  0.2605  0.8916  0.6331  0.8922
valid    33  0.2887  0.8611  0.5739  0.8817
test     34  0.2744  0.8847  0.5719  0.8817
train    43  0.2605  0.8916  0.6331  0.8922
valid    33  0.2887  0.8611  0.5739  0.8817
test     37  0.2744  0.8847  0.5719  0.8817
train    43  0.2605  0.8916  0.6331  0.8922
valid    33  0.2887  0.8611  0.5739  0.8817
test     34  0.2744  0.8847  0.5719  0.8817
02-08-21 11:57:[Eval]
02-08-21 11:58:[EDUTEM Epoch 0]
02-08-21 11:58:[train] bceloss:0.2605 auroc:0.8916 auprc:0.6331 acc:0.8922 
02-08-21 11:58:[valid] bceloss:0.2887 auroc:0.8611 auprc:0.5739 acc:0.8817 
02-08-21 11:58:[test] bceloss:0.2744 auroc:0.8847 auprc:0.5719 acc:0.8817 
